In [ ]:
import os

from dotenv import load_dotenv
from openai import OpenAI, AzureOpenAI
from rich import print as pprint
import tiktoken

#### 1. Load the environment variables.

In [ ]:
load_dotenv()

True

#### 2. Construct an instance of OpenAI.

In [ ]:
# client = OpenAI(
#   api_key = os.getenv("OPENAI_API_KEY"),
# )

client = AzureOpenAI(
    api_version=os.getenv("OPENAI_API_VERSION"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

#### 3. Making an API request.

In [ ]:
system_prompt = "請你用文言文的方式回答使用者的問題。"
user_prompt = "請推薦我今天午餐要吃什麼？"

completion = client.chat.completions.create(
  model="gpt-35-turbo-120",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ]
)

print(f'Prompt: {user_prompt}')
print(f'Completion: {completion.choices[0].message.content}')

Prompt: 請推薦我今天午餐要吃什麼？
Completion: 士子：蒙吾尊詢，午膳之事，實為小生恐難為之也。蓋今天世事繁衍，網羅萬象，各種美食均可供應。君欲何種菜餚或飲食之物？方可明智布置。舉凡糙米飯、湯羹品、蔬菜果品、肉魚蛋類，皆為膳食之佳選。休者之間，亦可杯羹小酌，以添娛樂之意。

然衣食住行，各有所好。捷足先登者或有所雅好，倘欲嘗鮮新潮，若差一些不是風尚者，亦未可知。但願君斟酌良久，懷思遐情，方可作出最佳選擇。


In [ ]:
# Rich text and beautiful formatting in the terminal.
pprint(completion)

ChatCompletion(
    id='chatcmpl-9nM9VEiB18F0Hxl3R7NY43tIPyUDY',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='士子：蒙吾尊詢，午膳之事，實為小生恐難為之也。蓋今天世事繁衍，網羅萬象，各種美食均可供應。
君欲何種菜餚或飲食之物？方可明智布置。舉凡糙米飯、湯羹品、蔬菜果品、肉魚蛋類，皆為膳食之佳選。休者之間，亦可杯羹小
酌，以添娛樂之意。\n\n然衣食住行，各有所好。捷足先登者或有所雅好，倘欲嘗鮮新潮，若差一些不是風尚者，亦未可知。但願
君斟酌良久，懷思遐情，方可作出最佳選擇。',
                role='assistant',
                function_call=None,
                tool_calls=None
            ),
            content_filter_results={
                'hate': {'filtered': False, 'severity': 'safe'},
                'self_harm': {'filtered': False, 'severity': 'safe'},
                'sexual': {'filtered': False, 'severity': 'safe'},
                'violence': {'filtered': False, 'severity': 'safe'}
            }
        )
    ],
    created=1721550253,
    model='gpt-35-turbo',
    object='chat.completion',
    service_tier=None,
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=283, prompt_tokens=54, total_tokens=337),
    prompt_filter_results=[
        {
            'prompt_index': 0,
            'content_filter_results': {
                'hate': {'filtered': False, 'severity': 'safe'},
                'self_harm': {'filtered': False, 'severity': 'safe'},
                'sexual': {'filtered': False, 'severity': 'safe'},
                'violence': {'filtered': False, 'severity': 'safe'}
            }
        }
    ]
)

### Token
- Token 通常指的是文字處理過程中的最小單位。 
- Token 是模型視角中的單字，可以是字符、詞語、片語、句子或其他較小的文字單元，取決於模型的設計。
- Tokenization 是將一段連續的文字序列拆分為 Token 的過程。

- Embedding 是將離散的文字資料轉換為連續的、低維度向量。
- Embedding 是把人類的語言轉換成電腦看得懂的語言或意思。

In [ ]:
encoder = tiktoken.encoding_for_model('gpt-3.5-turbo')

In [ ]:
encoder = tiktoken.encoding_for_model('gpt-3.5-turbo')

tokens = encoder.encode('我愛你')
print(f'人類視角看到的文字：{encoder.decode(tokens)}')
print(f'模型視角看到的向量：{tokens}')
print('---')

tokens = encoder.encode('我喜歡你')
print(f'人類視角看到的文字：{encoder.decode(tokens)}')
print(f'模型視角看到的向量：{tokens}')


人類視角看到的文字：我愛你
模型視角看到的向量：[37046, 31374, 249, 57668]
---
人類視角看到的文字：我喜歡你
模型視角看到的向量：[37046, 83601, 250, 15722, 94, 57668]


#### Q: 為什麼使用 ChatGPT API 跟用 TikToken Encoder 計算出來的 token 數量會不一樣呢？

In [ ]:
token_count = completion.usage.prompt_tokens
print(f'ChatGPT Token count: {token_count}')

encoder = tiktoken.encoding_for_model('gpt-3.5-turbo')
token_count = len(encoder.encode(system_prompt+user_prompt))
print(f'TikToken Token count: {token_count}')

ChatGPT Token count: 54
TikToken Token count: 43


#### A: 因為 prompt 的 Chat Markup Language (ChatML) 也要納入 token 數量的計算。

當 messages 輸入 ...
``` 
[
    {"role": "system", "content": system_prompt_content},
    {"role": "user", "content": user_prompt_content}
]
```
等同於 ...
```
<|im_start|>system\n{system_prompt_content}<|im_end|>   => 4 個 token 數
<|im_start|>user\n{user_prompt_content}<|im_end|>       => 4 個 token 數
<|im_start|>assistant<|message|>                        => 3 個 token 數
```
因此需要額外再加 11 個 token 數。


### ChatGPT API 參數使用與說明

- `n`: 控制 completion 輸出的數量

+ `max_tokens`: 控制 completion 的 token 最大上限，預設為模型上下文的最大長度
    - 輸入值值不可以超過 model 回傳輸出的上限（context_length_exceeded）
    - `finish_reason=length`，表示 completion 長度被限制而終止
    - `finish_reason=stop`，表示 completion 正常結束

- `temperature`: 控制 completion 的變化比重/隨機性，介於 0 到 2 之間，預設為 1
    - `temperature=0`，表示隨機性最低 = 回傳機率最高的下一個字
    - `temperature=1.5`，表示隨機性介於中間
    - `temperature=2`，表示隨機性最高，但會跑很久，不建議這樣設置
    - `seed`

+ `logit`: 控制 completion 的權重
    - `logit_bias`: 介於 -100(絕對不要出現) 到 100(絕對要出現) 之間，盡可能約束模型，但不能 100％ 控制
    - `logprobs`
    - `top_logprobs`

- `penalty`: 懲罰項，用以控制詞彙的重複性
    - `top_p`: 控制 completion 的變化比重，和 `temperature` 擇一使用，預設為 1
        - `top_p=0`，回傳最高的可能性，等價於 `temperature=0` 的效果
        - `top_p=1`，增加隨機性，等價於 `temperature=1` 的效果  
    - `presence_penalty`: 介於 -2 到 2 之間，預設為 0
        - `presence_penalty=2`，表示盡可能不出現重複的字詞
        - `presence_penalty=-2`，表示盡可能出現重複的字詞
    - `frequency_penalty`: 介於 -2 到 2 之間，預設為 0
        - 和 `presence_penalty` 作用一樣，只是算法不一樣

+ `response_format`: 指定 completion 回傳格式
    - `response_format={'type': 'text'}`
    - `response_format={'type': 'json_object'}`，需要在 system_prompt 寫 “請用 json 格式回覆”，此功能才會 work

Reference: https://platform.openai.com/docs/api-reference/chat/create

In [ ]:
encoder = tiktoken.encoding_for_model('gpt-3.5-turbo')
encoder.encode('吾我你他')

[7305, 122, 37046, 57668, 43511]

In [ ]:
system_prompt = "請用文言文的方式回答使用者的問題"
user_prompt = "你好～我叫黃小豬，是女生，請問你吃飽了嗎？"

completion = client.chat.completions.create(
  model="gpt-35-turbo-120",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ],
  n=3,
  # max_tokens=40,
  # temperature=1.5,
  # seed=1,
  # logit_bias={7305: -100, 122: -100, 37046: -100, 57668: -100, 43511: -100},
  # top_p=0,
  # presence_penalty=2,
  # response_format={'type': 'json_object'},
)

print(f'建議: {system_prompt}')
print(f'問題: {user_prompt}')
print(f'實際使用的模型: {completion.model}')
print(f'總共使用的 Token 數: {completion.usage.total_tokens} = {completion.usage.prompt_tokens} (問題) + {completion.usage.completion_tokens} (回覆)')
print('='* 30 )
for i, choice in enumerate(completion.choices):
    print(f'回覆 {i+1}: {choice.message.content}')
    print(f'回覆 {i+1} 終止的原因: {choice.finish_reason}')
    print('-' * 30)


建議: 請用文言文的方式回答使用者的問題
問題: 你好～我叫黃小豬，是女生，請問你吃飽了嗎？
實際使用的模型: gpt-35-turbo
總共使用的 Token 數: 376 = 59 (問題) + 317 (回覆)
回覆 1: 恭敬問候黃小豬女士，卑職已經用膳過了，謝謝您的關心。敢問女士飲食是否安排妥當？
回覆 1 終止的原因: stop
------------------------------
回覆 2: 黃小豬小姐，您好！臣妾衷心感謝您的關心，但是作為一個語言模型AI，臣妾無法進食，亦無需進食，只有為您提供文字上的協助和回答問題的能力。若有其他問題，請隨時提出，臣妾將竭誠為您服務。
回覆 2 終止的原因: stop
------------------------------
回覆 3: None
回覆 3 終止的原因: content_filter
------------------------------
